# Homework 3 02 - 712

## Problem Set 1

In [ ]:
!pip install python-docx
import docx

# Replace 'r' with 'rb' to open the file in binary read mode
# Use the docx library to extract the text from the file
doc = docx.Document('/content/drive/MyDrive/PS3test1_HW3.docx')
lines = [paragraph.text for paragraph in doc.paragraphs]

# Extracting the variables
k10 = float(lines[0].strip())
k20 = float(lines[1].strip())
delta_k = float(lines[2].strip())
delta_t = float(lines[3].strip())
n = int(lines[4].strip())
r = int(lines[5].strip())

# Extracting the matrix Y (3 rows of data)
Y = []
for i in range(6, 9):
    row = list(map(float, lines[i].split()))
    Y.append(row)

Converting Y to Matrix

In [ ]:
import numpy as np
Y_matrix = np.array(Y)
Y_matrix

array([[1.       , 0.9789513, 1.116627 , 0.7190759, 1.0454412, 0.8212455,
        0.8289814, 0.9623663, 0.9656249, 0.8103866, 0.7251547],
       [2.       , 1.8863545, 2.0811257, 2.0121596, 2.0022361, 1.962572 ,
        1.8684818, 1.8766915, 1.9787224, 1.9564449, 2.0084744],
       [0.5      , 0.5726863, 0.5197492, 0.145477 , 0.6390091, 0.3752911,
        0.5748619, 0.3890326, 0.534364 , 0.4486411, 0.5268894]])

## Defining Φ(k1, k2) using elements from file

In [ ]:
def Φ(x, y):
    # Initialize the matrix X with 3 rows for A, B, C, and n+1 columns for each time step
    X = np.zeros(Y_matrix.shape)

    # Set the initial concentrations
    X[0, 0] = 1#Y_matrix[0][0]
    X[1, 0] = 2#Y_matrix[1][0]
    X[2, 0] = 1/2#Y_matrix[2][0]

    # Iterate over the number of time steps
    for i in range(10):
        A_prev = X[0, i]
        B_prev = X[1, i]
        C_prev = X[2, i]

        # Apply Forward Euler method to update concentrations
        dA = -2 * x * A_prev**2 * B_prev + y * C_prev
        dB = -x * A_prev**2 * B_prev + y * C_prev
        dC = x * A_prev**2 * B_prev - y * C_prev

        # Update concentrations for the next time step
        X[0, i+1] = A_prev + delta_t * dA
        X[1, i+1] = B_prev + delta_t * dB
        X[2, i+1] = C_prev + delta_t * dC
        #print(X)

    # Return the concentration matrix X
    def least_squares(A, B):
     error = 0
     for i in range(len(A)):
       for j in range(len(A[i])):
         error += (A[i][j] - B[i][j])**2
     return error
    return least_squares(X, Y_matrix)

Proving that Φ(k1, k2) works:

In [ ]:
Φ(k10, k20)

0.3926792420433009

### Gradient function for Φ (1st order approximation):

In [ ]:
def Gradient(x, y, h):
    G = np.zeros(2)

    def delta_Φ_delta_x(x, y, h):
        return (Φ(x + h, y) - Φ(x, y)) / h  # 1st order approximation

    def delta_Φ_delta_y(x, y, h):
        return (Φ(x, y + h) - Φ(x, y)) / h  # 1st order approximation

    G[0] = delta_Φ_delta_x(x, y, h)
    G[1] = delta_Φ_delta_y(x, y, h)

    return G

Gradient works

In [ ]:
Gradient(k10,k20, delta_k)

array([ 1.0982354 , -0.33407375])

### Inverse Hessian function for Φ (1st order approximation):

In [ ]:
def Hessian_inv(x,y,h):
  H = np.zeros((2,2))
  def delsq_Φ_delta_xsq(x,y,h):
    return (Φ(x+h,y) - 2*Φ(x,y) + Φ(x-h,y))/h**2 # 1st order approximation

  def delsq_Φ_delta_ysq(x,y,h):
    return (Φ(x,y+h) - 2*Φ(x,y) + Φ(x,y-h))/h**2 # 1st order approximation

  def delsq_Φ_delta_xy(x,y,h):
    return (Φ(x+h,y+h) - Φ(x+h,y) - Φ(x,y+h) + Φ(x,y))/h**2 # 1st order approximation
  H[0][0] = delsq_Φ_delta_xsq(x,y,h)
  H[0][1] = delsq_Φ_delta_xy(x,y,h)
  H[1][0] = delsq_Φ_delta_xy(x,y,h) # Assuming Φ is a smooth function
  H[1][1] = delsq_Φ_delta_ysq(x,y,h)
  return np.linalg.inv(H)

Hessian_inv works

In [ ]:
Hessian_inv(k10,k20, delta_k)

array([[0.08260137, 0.31677635],
       [0.31677635, 1.43816482]])

## First Iteration

In [ ]:
def func(x,y) :
  h = delta_k
  F = np.zeros(2)
  Dot = np.dot(Hessian_inv(x,y,h), Gradient(x,y,h))
  F[0] = x - Dot[0]
  F[1] = y - Dot[1]
  return F

func(k10,k20)

array([0.09511091, 0.28255811])

In [ ]:
def k_final(iterations):
  k1 = k10
  k2 = k20
  print(f"Iteration 0: k1 = {k10}, k2 = {k20}, Φ = {Φ(k10,k20)}")
  for i in range(iterations):
    F = func(k1,k2)
    k1 = F[0]
    k2 = F[1]

    print(f"Iteration {i+1}: k1 = {k1}, k2 = {k2}, Φ = {Φ(k1,k2)}")
  return print(f"Final values: k1 = {F[0]}, k2 = {F[1]}, Φ = {Φ(k1,k2)}")

Solution to Problem Set 3 Test 1

In [ ]:
k_final(r)

Iteration 0: k1 = 0.08, k2 = 0.15, Φ = 0.3926792420433009
Iteration 1: k1 = 0.09511091489331142, k2 = 0.2825581073685409, Φ = 0.37742550538131686
Iteration 2: k1 = 0.09849171379277853, k2 = 0.30939798247862943, Φ = 0.37703258426986475
Iteration 3: k1 = 0.09858819189448424, k2 = 0.3102120018733302, Φ = 0.3770317505945494
Iteration 4: k1 = 0.09858885098267861, k2 = 0.31021534398983863, Φ = 0.3770317476378006
Iteration 5: k1 = 0.09858885464106074, k2 = 0.31021536043389164, Φ = 0.3770317476217628
Iteration 6: k1 = 0.09858885466228624, k2 = 0.310215360529771, Φ = 0.37703174762166985
Iteration 7: k1 = 0.09858885466347722, k2 = 0.31021536053558696, Φ = 0.3770317476216644
Iteration 8: k1 = 0.09858885466283668, k2 = 0.31021536053240767, Φ = 0.3770317476216673
Iteration 9: k1 = 0.09858885466296792, k2 = 0.31021536053312265, Φ = 0.3770317476216665
Iteration 10: k1 = 0.09858885466224511, k2 = 0.31021536052959864, Φ = 0.3770317476216698
Final values: k1 = 0.09858885466224511, k2 = 0.310215360529598

## Problem Set 2

In [ ]:
doc2 = docx.Document('/content/drive/MyDrive/PS3test2_HW3.docx')
lines = [paragraph.text for paragraph in doc2.paragraphs]

# Extracting the variables
k10 = float(lines[0].strip())
k20 = float(lines[1].strip())
delta_k = float(lines[2].strip())
delta_t = float(lines[3].strip())
n = int(lines[4].strip())
r = int(lines[5].strip())

# Extracting the matrix Y (3 rows of data)
Y = []
for i in range(6, 9):
    row = list(map(float, lines[i].split()))
    Y.append(row)

Converting Y to Matrix

In [ ]:
Y_matrix = np.array(Y)
Y_matrix

array([[2.       , 1.651548 , 1.422473 , 1.366318 , 1.118542 , 1.222559 ,
        0.9108657, 1.030183 , 0.9442344, 0.9133284, 0.8545681],
       [1.5      , 1.321081 , 1.133595 , 1.021138 , 1.049023 , 1.319359 ,
        1.0686325, 1.096857 , 0.9462353, 0.8370607, 0.9202124],
       [1.       , 1.207463 , 1.384705 , 1.293249 , 1.485911 , 1.425603 ,
        1.4588016, 1.575164 , 1.5798663, 1.6510563, 1.5259495]])

## Defining Φ(k1, k2) using elements from file

In [ ]:
def Φ(k1, k2):
    # Initialize the matrix X with 3 rows for A, B, C, and n+1 columns for each time step
    X = np.zeros(Y_matrix.shape)

    # Set the initial concentrations
    X[0, 0] = 2 #Y_matrix[0][0]
    X[1, 0] = 1 #Y_matrix[1][0]
    X[2, 0] = 1 #Y_matrix[2][0]

    # Iterate over the number of time steps
    for i in range(10):
        A_prev = X[0, i]
        B_prev = X[1, i]
        C_prev = X[2, i]

        # Apply Forward Euler method to update concentrations
        dA = -2 * k1 * A_prev**2 * B_prev + k2 * C_prev
        dB = -k1 * A_prev**2 * B_prev + k2 * C_prev
        dC = k1 * A_prev**2 * B_prev - k2 * C_prev

        # Update concentrations for the next time step
        X[0, i+1] = A_prev + delta_t * dA
        X[1, i+1] = B_prev + delta_t * dB
        X[2, i+1] = C_prev + delta_t * dC

    # Return the concentration matrix X
    def least_squares(X, Y):
     error = 0
     for i in range(len(X)):
       for j in range(len(X[i])):
         error += (X[i][j] - Y[i][j])**2
     return error
    return least_squares(X, Y_matrix)

In [ ]:
Φ(k10, k20)

2.2699417462092466

### Gradient function and Inverse Hessian function for Φ (1st order approximation):

In [ ]:
def Gradient(x, y, h):
    G = np.zeros(2)

    def delta_Φ_delta_x(x, y, h):
        return (Φ(x + h, y) - Φ(x, y)) / h  # 1st order approximation

    def delta_Φ_delta_y(x, y, h):
        return (Φ(x, y + h) - Φ(x, y)) / h  # 1st order approximation

    G[0] = delta_Φ_delta_x(x, y, h)
    G[1] = delta_Φ_delta_y(x, y, h)

    return G

def Hessian_inv(x,y,h):
  H = np.zeros((2,2))
  def delsq_Φ_delta_xsq(x,y,h):
    return (Φ(x+h,y) - 2*Φ(x,y) + Φ(x-h,y))/h**2 # 1st order approximation

  def delsq_Φ_delta_ysq(x,y,h):
    return (Φ(x,y+h) - 2*Φ(x,y) + Φ(x,y-h))/h**2 # 1st order approximation

  def delsq_Φ_delta_xy(x,y,h):
    return (Φ(x+h,y+h) - Φ(x+h,y) - Φ(x,y+h) + Φ(x,y))/h**2 # 1st order approximation
  H[0][0] = delsq_Φ_delta_xsq(x,y,h)
  H[0][1] = delsq_Φ_delta_xy(x,y,h)
  H[1][0] = delsq_Φ_delta_xy(x,y,h) # Assuming Φ is a smooth function
  H[1][1] = delsq_Φ_delta_ysq(x,y,h)
  return np.linalg.inv(H)

## First Iteration

In [ ]:
def func(x,y) :
  h = delta_k
  F = np.zeros(2)
  Dot = np.dot(Hessian_inv(x,y,h), Gradient(x,y,h))
  F[0] = x - Dot[0]
  F[1] = y - Dot[1]
  return F

func(k10,k20)

array([0.49336473, 0.40280533])

In [ ]:
import pandas as pd
def k_final(iterations):
  df = pd.DataFrame(columns=['Iteration', 'k1', 'k2', 'Φ'])
  k1 = k10
  k2 = k20
  print(f"Iteration 0: k1 = {k10}, k2 = {k20}, Φ = {Φ(k10,k20)}")
  for i in range(iterations):
    F = func(k1,k2)
    k1 = F[0]
    k2 = F[1]

    print(f"Iteration {i+1}: k1 = {k1}, k2 = {k2}, Φ = {Φ(k1,k2)}")
    df.loc[i] = [i+1, k1, k2, Φ(k1,k2)]
  print(f"Final values: k1 = {F[0]}, k2 = {F[1]}, Φ = {Φ(k1,k2)}")
  return df

Solution to Problem Set 3 Test 2

In [ ]:
df = k_final(r)

Iteration 0: k1 = 0.4, k2 = 0.3, Φ = 2.2699417462092466
Iteration 1: k1 = 0.49336472939274306, k2 = 0.40280533375813066, Φ = 2.182355568974309
Iteration 2: k1 = 0.521953027755336, k2 = 0.4217102071120364, Φ = 2.176405737341857
Iteration 3: k1 = 0.5245146535909586, k2 = 0.42337073648398915, Φ = 2.1763633962861197
Iteration 4: k1 = 0.5245344984095319, k2 = 0.4233844458357548, Φ = 2.176363367996024
Iteration 5: k1 = 0.5245345149709835, k2 = 0.423384467971657, Φ = 2.176363367969794
Iteration 6: k1 = 0.5245345149803492, k2 = 0.42338446798387724, Φ = 2.1763633679697807
Iteration 7: k1 = 0.524534514986043, k2 = 0.42338446799192964, Φ = 2.176363367969772
Iteration 8: k1 = 0.524534514985063, k2 = 0.42338446799028745, Φ = 2.1763633679697736
Iteration 9: k1 = 0.524534514982917, k2 = 0.42338446798520246, Φ = 2.1763633679697754
Iteration 10: k1 = 0.524534514981664, k2 = 0.4233844679848399, Φ = 2.1763633679697776
Final values: k1 = 0.524534514981664, k2 = 0.4233844679848399, Φ = 2.1763633679697776


In [ ]:
df

,Iteration,k1,k2,Φ
0,1.0,0.493365,0.402805,2.182356
1,2.0,0.521953,0.421710,2.176406
2,3.0,0.524515,0.423371,2.176363
3,4.0,0.524534,0.423384,2.176363
4,5.0,0.524535,0.423384,2.176363
5,6.0,0.524535,0.423384,2.176363
6,7.0,0.524535,0.423384,2.176363
7,8.0,0.524535,0.423384,2.176363
8,9.0,0.524535,0.423384,2.176363
9,10.0,0.524535,0.423384,2.176363
